# Test an organic molecules HOMO/LUMO and Transfer Integral With morphCT.

1. this notebooks seeks to display a simple workflow for using morphct functionalities to explore  the energetics of a given organic molecule. The energy level at the HOMO/LUMO is the foundation for everything morphct does. with that, veryfiying the accuracy of this calculation is a crucial first step when investigating a new molecule with morhct. 
2. In this notebook we will consider thiophene, which is the fundemental building block of P3HT (a benchmark electron donating compound. We will use to establish a worklow for comparing Morphct outputs with DFT or experimental results. However, this worklow should easily adapt to any organic compound whose energetic properties we might wish to investigate. 
3. Morphct models the charge mobility of a substance by considering where a charge may delocalize (on a chromophore) and modeling the rate at which that delocalized charge will hop to nieghboring molecules (or chromophores). Having done this for every chromophore in a morphology, morphct can run a simulation to model how well charges move around the environment.
4. Morphct models the rate at which a hop will take place between any two chromophores with semi classical Marcus Theory. To do so requires an accurate description of the driving force for electron transfer, and the orbital overlap of the two chromophores. Both of these can be estimated with the chromophores energy levels at the HOMO/LUMO (highest occupied molecular orbital/ lowest unoccupies molecular orbital). This estimation is a simplification for computational purposes, and it is, therefore, critical to verify that these assumptions match more rigorous theoretical techniques like DFT and experimental results (to whatever degree they are available). 
5. In short, we will take two thiophene rings to be two chromophores. Morphct will calculate the HOMO of both chomophores as well as the orbital overlap between them, as measured by the transfer integral (also estimated by the interaction of the two chromophores HOMOs). We will repeat this over thousands of angles and distances between the two rings.

In [ ]:
import os
import mbuild as mb
import gsd.hoomd
import hoomd
import matplotlib.pyplot as plt
from mbuild import packing
import itertools as it
import numpy as np
from morphct import execute_qcc as eqcc
from morphct import mobility_kmc as kmc
from morphct import chromophores
from morphct import kmc_analyze
from morphct.chromophores import conversion_dict
from morphct.chromophores import amber_dict


# Below we specify the smiles string for the molecule of interest (mbuild takes mol2 as well) and we use the mbuild fill_box to put two of these molecule in a box together with random orientations. We do this 100 times and save the outputs as gsds in the data/fillbox-gsds/ directory. Fill_box allows you to specifiy the density as well as the minimum distance apart that molecules should sit. In this case I have chosen .3nm, as this is what evans paper says is physically acceptable. 

MorphCT uses pySCF to do the quantum chemical calculations behind the above numbers. PYSCF uses a MINDO3 method to simplify the calculation. As you can see it only took seconds to calculate (go ahead and solve shrodingers in that time I dare you). The diagram below shows that the numbers are comparable to the more rigorous DFT methods used in the work cited below. 

In [ ]:
# https://doi.org/10.1016/j.molliq.2017.05.020
from IPython import display
display.Image("thiophene.jpg")

# TRANSFER INTEGRALS
Now we can get the transfer integral for our gsds. 

 That gives a sample of randomly oreinted thiophenes. Printing the average vs the max
 to get an understanding of the shape of the data. in the supplimental info of this paper
 (https://www.mdpi.com/2073-4360/10/12/1358) it says TI variation within a factor of 2-3 
 is not expected to effect the mobility calculation and this is within those bounds

In [ ]:
heat_tuple_list = []
transfer_integral_list = []
path = "/Users/jimmy/cmelab/data/TI/angled"
if not os.path.isdir(path):
    os.makedirs(path)
smi = "C1=CSC=C1"
thiophene = mb.load(smi, smiles=True)
distances = np.linspace(0, 0.5, 10)
angles = np.linspace(0, np.pi/2, 10) # rotate around the sulfur axis (x-axis)
# or pitch although i cant understand why the sulfur isnt the cockpit
snap_list = []
#create a product space of distances and angles as delineated above. get two distances
# and two angles. assuming distance is a distance from the origin the if the distances from
# they are on top of each other and you can skip that one as in line 13
for d1, a1, d2, a2 in it.product(distances, angles, repeat=2):
    name = f"thiophene_d-{d1:.1f}-{d2:.1f}_a-{a1:.1f}-{a2:.1f}.gsd"
    if d1 == 0 and d2 == 0:
        continue
    #trying to say if the c - c distance is less than .3nm from evans paper 
    #skip they are too close for real life. and above .5 the TI is too weak
    #just from o
    if np.sqrt(d1**2+d2**2) < .3 or np.sqrt(d1**2+d2**2) > .5:
        continue
    system = mb.Compound()
    system.add(mb.clone(thiophene), "1")
    system.add(mb.clone(thiophene), "2")
    system["2"].rotate(a1, [0,0,1]) 
    system["2"].rotate(a2,[1,0,0])
    system["2"].translate([d1,0,d2])
    system.save(path+name, overwrite=True)
    gsd_file = path + name
    with gsd.hoomd.open(name=gsd_file, mode='rb') as f:
        snap = f[0]
        
    ref_distance = 1
    
    snap.particles.position *= ref_distance
    snap.configuration.box[:3] *= ref_distance
    chromo_ids = np.array([0,1,2,3,4,5,6,7,8,])
    chromo2_ids = np.array([9,10,11,12,13,14,15,16,17])
    aaids=[]
    aaids.append(chromo_ids)
    aaids.append(chromo2_ids)
    
    chromo_list = []
    for j,aaid in enumerate(aaids):
        chromo_list.append(chromophores.Chromophore(j, snap, aaid, "donor", amber_dict))
        
    qcc_pairs = chromophores.set_neighbors_voronoi(chromo_list, snap, amber_dict, d_cut=100)
    
    dimer_data = eqcc.dimer_homolumo(qcc_pairs, chromo_list, path +"two-molecule-test.txt")
    
    data = eqcc.singles_homolumo(chromo_list, path + "singles.txt")
    
    eqcc.set_energyvalues(chromo_list, path + "singles.txt", path + "two-molecule-test.txt")
    
    chromo =chromo_list[1]
    #dont know if ill need this or if the math is right
    #ss_dist= np.sqrt((d1*np.cos(a1))**2+(d1*np.sin(a1))**2+d2**2)
    cc_dist = np.sqrt(d1**2+d2**2)
    heat_tuple_list.append([cc_dist, a2 , chromo.neighbors_ti[0]])
    
    transfer_integral_list.append(chromo.neighbors_ti[0])

In [ ]:
import seaborn as sns
x = [i[0] for i in heat_tuple_list]
y = [i[1] for i in heat_tuple_list]
z = [i[2] for i in heat_tuple_list]
ax = plt.axes(projection='3d')
ax.scatter(x, y, z, c=z, cmap='viridis', linewidth=0.5);

In [ ]:
#import pickle
#system_pickle = open('transfer_integral_list.pickle','wb')
#pickle.dump(transfer_integral_list, system_pickle)
#system_pickle.close()

In [ ]:
#import seaborn as sns
x = [i[0]*10 for i in heat_tuple_list]
y = [i[1] for i in heat_tuple_list]
z = [i[2] for i in heat_tuple_list]
fig = plt.figure(figsize = (10, 10))
ax = plt.axes(projection='3d')
plt.xticks(fontsize=14)
plt.yticks(fontsize=14)

ax.set_xlabel('Center-to-center distance (Angstrom)',fontdict=dict(weight='bold'),fontsize=16, labelpad = 12)
ax.set_ylabel('Pitch (Radians)',fontdict=dict(weight='bold'),fontsize=16, labelpad = 12)
ax.set_zlabel('TI (eV)',fontdict=dict(weight='bold'),fontsize=16, labelpad = 12)
for t in ax.zaxis.get_major_ticks(): t.label.set_fontsize(14)
#ax.set_title('Thiophene Transfer Integrals')
p = ax.scatter(x, y, z, c=z, cmap='rainbow', linewidth=7);
#plt.colorbar(p, pad = .1, aspect = 2.3)



fig.show()

In [ ]:
fig.savefig('../src/figures/transfer_integral_plot', bbox_inches = "tight", facecolor = "white")

In [ ]:
import pickle

In [ ]:
file = open('cc_pitch_ti-tuple.pickle','rb')
thios = pickle.load(file)

In [ ]:
#system_pickle = open('cc_pitch_ti-tuple.pickle','wb')
#pickle.dump(heat_tuple_list, system_pickle)
#system_pickle.close()

In [ ]:
print(np.mean(thios))

In [ ]:
cc_restrict = []
for i in heat_tuple_list:
    if i[0] < .4 and i[0] > .38 and i[1] <.05:
        cc_restrict.append(i[2])

In [ ]:
len(cc_restrict)

In [ ]:
np.mean(heat_tuple_list[0])


In [ ]:
np.std(heat_tuple_list[0])

In [ ]:
heat_tuple_list


In [ ]:
small_heat_tuple_list = []
small_transfer_integral_list = []
path = "/Users/jimmy/cmelab/data/TI/angled"
if not os.path.isdir(path):
    os.makedirs(path)
smi = "C1=CSC=C1"
thiophene = mb.load(smi, smiles=True)


snap_list = []


name = "3.8angs.gsd"

system = mb.Compound()
system.add(mb.clone(thiophene), "1")
system.add(mb.clone(thiophene), "2")


system["2"].translate([0,0,.4])
system.save(path+name, overwrite=True)
gsd_file = path + name
with gsd.hoomd.open(name=gsd_file, mode='rb') as f:
    snap = f[0]

ref_distance = 1

snap.particles.position *= ref_distance
snap.configuration.box[:3] *= ref_distance
chromo_ids = np.array([0,1,2,3,4,5,6,7,8,])
chromo2_ids = np.array([9,10,11,12,13,14,15,16,17])
aaids=[]
aaids.append(chromo_ids)
aaids.append(chromo2_ids)

chromo_list = []
for j,aaid in enumerate(aaids):
    chromo_list.append(chromophores.Chromophore(j, snap, aaid, "donor", amber_dict))

qcc_pairs = chromophores.set_neighbors_voronoi(chromo_list, snap, amber_dict, d_cut=100)

dimer_data = eqcc.dimer_homolumo(qcc_pairs, chromo_list, path +"two-molecule-test.txt")

data = eqcc.singles_homolumo(chromo_list, path + "singles.txt")

eqcc.set_energyvalues(chromo_list, path + "singles.txt", path + "two-molecule-test.txt")

chromo =chromo_list[1]
#dont know if ill need this or if the math is right
#ss_dist= np.sqrt((d1*np.cos(a1))**2+(d1*np.sin(a1))**2+d2**2)

small_heat_tuple_list.append(chromo.neighbors_ti[0])

In [ ]:
small_heat_tuple_list

In [ ]:
thios[1408]

In [ ]:
x = 0
index = 0
for i, thio in enumerate(thios):
    if thio[2] > x:
        x = thio[2]
        index = i

In [ ]:
x

In [ ]:
index

In [ ]:
%%time
heat_tuple_list = []
transfer_integral_list = []
path = "TI-04-2022-new/"
if not os.path.isdir(path):
    os.makedirs(path)
smi = "C1=CSC=C1"
thiophene = mb.load(smi, smiles=True)
distances = np.linspace(0, 0.5, 12)
angles = np.linspace(-np.pi/2, 0, 12) # rotate around the sulfur axis (x-axis)
# or pitch although i cant understand why the sulfur isnt the cockpit
snap_list = []
#create a product space of distances and angles as delineated above. get two distances
# and two angles. assuming distance is a distance from the origin the if the distances from
# they are on top of each other and you can skip that one as in line 13
for d1, a1, d2, a2 in it.product(distances, angles, repeat=2):
    name = f"thiophene_d-{d1:.1f}-{d2:.1f}_a-{a1:.1f}-{a2:.1f}.gsd"
    system = mb.Compound()
    system.add(mb.clone(thiophene), "1")
    system.add(mb.clone(thiophene), "2")
    system["2"].rotate(a2,[1,0,0])
    system["2"].rotate(a1, [0,0,1])
    system["2"].translate([d1,0,d2])
    bottom_atoms = system["1"].xyz
    top_atoms = system["2"].xyz
    min_dist = 10
    #calculate distance between atoms of different ring and save the minimum distance
    for atom in bottom_atoms:
        for x in top_atoms:
            if np.linalg.norm(atom - x) < min_dist:
                min_dist = np.linalg.norm(atom - x)
    print(min_dist)
    #if any of the atoms are less that 3 angstoms apart we dont create the gsd
    if min_dist > .3:
        system.save(path+name, overwrite=True)
        gsd_file = path + name
        with gsd.hoomd.open(name=gsd_file, mode='rb') as f:
            snap = f[0]
        ref_distance = 1
        snap.particles.position *= ref_distance
        snap.configuration.box[:3] *= ref_distance
        chromo_ids = np.array([0,1,2,3,4,5,6,7,8,])
        chromo2_ids = np.array([9,10,11,12,13,14,15,16,17])
        aaids=[]
        aaids.append(chromo_ids)
        aaids.append(chromo2_ids)
        chromo_list = []
        for j,aaid in enumerate(aaids):
            chromo_list.append(chromophores.Chromophore(j, snap, aaid, "donor", amber_dict))
        qcc_pairs = chromophores.set_neighbors_voronoi(chromo_list, snap, amber_dict, d_cut=100)
        dimer_data = eqcc.dimer_homolumo(qcc_pairs, chromo_list, path +"two-molecule-test.txt")
        data = eqcc.singles_homolumo(chromo_list, path + "singles.txt")
        eqcc.set_energyvalues(chromo_list, path + "singles.txt", path + "two-molecule-test.txt")
        chromo =chromo_list[1]
        #dont know if ill need this or if the math is right
        #ss_dist= np.sqrt((d1*np.cos(a1))**2+(d1*np.sin(a1))**2+d2**2)
        cc_dist = np.sqrt(d1**2+d2**2)
        heat_tuple_list.append([cc_dist, a1, a2 , d1, d2, chromo.neighbors_ti[0]])
        transfer_integral_list.append(chromo.neighbors_ti[0])

In [ ]:
system_pickle = open('cc_a1_a2_d1_d2_ti-tuple.pickle','wb')
pickle.dump(heat_tuple_list, system_pickle)
system_pickle.close()

In [ ]:
heat_tuple_list[1408]

In [ ]:
[i[0] for i in heat_tuple_list]

In [ ]:
heat_tuple_list